# Open Earth Foundation
## Open Ocean

This is the third notebook of a series of 5 notebooks that explains step by step how to calculate each modulating factor and assign credtis for the [Marine Biodiversity Credits methodology](https://zenodo.org/records/10182712) applied to the Cocos Marine Conservation Area of Costa Rica

## Libraries

In [ ]:
import os
import glob
import boto3

import math
import numpy as np
import pandas as pd
from pyobis import occurrences

import matplotlib.pyplot as plt

import shapely
import geopandas as gpd
from shapely.geometry import Polygon, Point, box
from shapely.ops import linemerge, unary_union, polygonize

In [ ]:
import fiona; #help(fiona.open)

In [ ]:
%load_ext autoreload

In [ ]:
#Run this to reload the python file
%autoreload 2
from MBU_utils import *

## Data

Two data sources are used for these calculations, data from OBIS (observation points) and those from the IUCN red list (distribution polygons).

- Indices such as Shannon and Simpson can be calculated only with OBIS data
- Metrics like WEGE and Endemism can be calculated only with IUCN data
- Habitats accounting is calculating with IUCN Global Ecosystem Typology

**Accessing OBIS data**

OBIS data contains species information and points of observation

In [ ]:
# MPA file
ACMC = gpd.read_file('https://ocean-program.s3.amazonaws.com/data/raw/MPAs/ACMC.geojson')

In [ ]:
# create a polygon to access the OBIS data
min_x, min_y, max_x, max_y = ACMC.total_bounds
geometry = f"POLYGON(({max_x} {min_y}, {min_x} {min_y}, {min_x} {max_y}, {max_x} {max_y}, {max_x} {min_y}))"

query = occurrences.search(geometry=geometry)
query.execute()

# returns the data
df2 = query.data 

In [ ]:
# convert OBIS dataframe to geodataframe
# making point with the lat and lon information
df2 = gpd.GeoDataFrame(df2, geometry=gpd.points_from_xy(df2.decimalLongitude, df2.decimalLatitude))

**Accesing IUCN RedList data**

In [ ]:
df1 = gpd.read_file('s3://ocean-program/data/processed/ACMC_IUCN_RedList/gdf_ACMC_IUCN_range_status_filtered.shp')

**Accesing IUCN Global Ecosystem Typology**

For this step, it's needed only the ubication path of these files

In [ ]:
# from AWS
s3 = boto3.resource('s3')
bucket = s3.Bucket('ocean-program')

eco_names = []

# list of all ecosystem layes
for obj in bucket.objects.filter(Prefix='data/raw/Ecosystems/'):
    name = obj.key[:]
    eco_names.append(f'https://ocean-program.s3.amazonaws.com/{name}')

Grid geodataframe

In [ ]:
# making a grid geoDataFrame
grid = create_grid(ACMC, grid_shape="hexagon", grid_size_deg=0.05)

# Modulating Factors

**Shannon Index**

In [ ]:
MF1 = shannon(ACMC, df2, grid)

In [ ]:
MF1.head()

In [ ]:
MF1 = gpd.clip(MF1.set_crs(epsg=4326, allow_override=True), ACMC.set_crs(epsg=4326, allow_override=True))

In [ ]:
ax = MF1.plot(column='shannon', figsize=(12, 10), cmap='viridis', edgecolor="grey", legend = True)
ax.yaxis.set_ticks([])
ax.xaxis.set_ticks([])

gpd.GeoSeries(ACMC.geometry).plot(ax=ax, edgecolor='black', facecolor='none')
plt.title('Modulating Factor: Shannon Index')

**Simpsion Index**

In [ ]:
MF2 = simpson(ACMC, df2, grid)

In [ ]:
MF2.head()

In [ ]:
MF2 = gpd.clip(MF2.set_crs(epsg=4326, allow_override=True), ACMC.set_crs(epsg=4326, allow_override=True))

In [ ]:
ax = MF2.plot(column='simpson', figsize=(12, 10), cmap='viridis', edgecolor="grey", legend = True)
ax.yaxis.set_ticks([])
ax.xaxis.set_ticks([])

gpd.GeoSeries(ACMC.geometry).plot(ax=ax, edgecolor='black', facecolor='none')
plt.title('Modulating Factor: Simpson Index')

**Species Richness**

In [ ]:
MF3 = species_richness(ACMC, df2, grid)

In [ ]:
MF3.head()

In [ ]:
MF3 = gpd.clip(MF3.set_crs(epsg=4326, allow_override=True), ACMC.set_crs(epsg=4326, allow_override=True))

In [ ]:
ax = MF3.plot(column='species_richness', figsize=(12, 10), cmap='viridis', edgecolor="grey", legend = True)
ax.yaxis.set_ticks([])
ax.xaxis.set_ticks([])

gpd.GeoSeries(ACMC.geometry).plot(ax=ax, edgecolor='black', facecolor='none')
plt.title('Modulating Factor: Species Richness')

**Endemism**

In [ ]:
MF4 = endemism(ACMC, df1, grid)

In [ ]:
MF4.head()

In [ ]:
MF4 = gpd.clip(MF4.set_crs(epsg=4326, allow_override=True), ACMC.set_crs(epsg=4326, allow_override=True))

In [ ]:
ax = MF4.plot(column='result', figsize=(12, 10), cmap='viridis', edgecolor="grey", legend = True)
ax.yaxis.set_ticks([])
ax.xaxis.set_ticks([])

gpd.GeoSeries(ACMC.geometry).plot(ax=ax, edgecolor='black', facecolor='none')
plt.title('Modulating Factor: Endemism')

**WEGE**

In [ ]:
MF5 = wege(ACMC, df1, grid)

In [ ]:
MF5.head()

In [ ]:
MF5 = gpd.clip(MF5.set_crs(epsg=4326, allow_override=True), ACMC.set_crs(epsg=4326, allow_override=True))

In [ ]:
ax = MF5.plot(column='result', figsize=(12, 10), cmap='viridis', edgecolor="grey", legend = True)
ax.yaxis.set_ticks([])
ax.xaxis.set_ticks([])

gpd.GeoSeries(ACMC.geometry).plot(ax=ax, edgecolor='black', facecolor='none')
plt.title('Modulating Factor: Wege')

**Habitats Accounting**

In [ ]:
MF6 = habitat_accounting(ACMC, grid, eco_names)

In [ ]:
MF6.head()

In [ ]:
MF6 = gpd.clip(MF6.set_crs(epsg=4326, allow_override=True), ACMC.set_crs(epsg=4326, allow_override=True))

In [ ]:
ax = MF6.plot(column='count', figsize=(12, 10), cmap='viridis', edgecolor="grey", legend = True)
ax.yaxis.set_ticks([])
ax.xaxis.set_ticks([])

gpd.GeoSeries(ACMC.geometry).plot(ax=ax, edgecolor='black', facecolor='none')
plt.title('Modulating Factor: Habitats Survey')